In [1]:
import time
import random
import math
people = [('Seymour','BOS'),
          ('Franny','DAL'),
          ('Zooey','CAK'),
          ('Walt','MIA'),
          ('Buddy','ORD'),
          ('Les','OMA')]
# LaGuardia airport in New York
destination='LGA'

In [2]:
flights = {}

for line in open('resources/schedule.txt'):
    origin,dest,depart,arrive,price=line.strip( ).split(',')
    flights.setdefault((origin,dest),[])
    flights[(origin,dest)].append([depart,arrive,price])

In [3]:
flights

{('BOS', 'LGA'): [['6:17', '8:26', '89'],
  ['8:04', '10:11', '95'],
  ['9:45', '11:50', '172'],
  ['11:16', '13:29', '83'],
  ['12:34', '15:02', '109'],
  ['13:40', '15:37', '138'],
  ['15:27', '17:18', '151'],
  ['17:11', '18:30', '108'],
  ['18:34', '19:36', '136'],
  ['20:17', '22:22', '102']],
 ('CAK', 'LGA'): [['6:08', '8:06', '224'],
  ['8:27', '10:45', '139'],
  ['9:15', '12:14', '247'],
  ['10:53', '13:36', '189'],
  ['12:08', '14:59', '149'],
  ['13:40', '15:38', '137'],
  ['15:23', '17:25', '232'],
  ['17:08', '19:08', '262'],
  ['18:35', '20:28', '204'],
  ['20:30', '23:11', '114']],
 ('DAL', 'LGA'): [['6:12', '10:22', '230'],
  ['7:53', '11:37', '433'],
  ['9:08', '12:12', '364'],
  ['10:30', '14:57', '290'],
  ['12:19', '15:25', '342'],
  ['13:54', '18:02', '294'],
  ['15:44', '18:55', '382'],
  ['16:52', '20:48', '448'],
  ['18:26', '21:29', '464'],
  ['20:07', '23:27', '473']],
 ('LGA', 'BOS'): [['6:39', '8:09', '86'],
  ['8:23', '10:28', '149'],
  ['9:58', '11:18', '13

In [4]:
def getminutes(t):
    x=time.strptime(t,'%H:%M')
    return x[3]*60+x[4]

In [5]:
def printschedule(r):
    for d in range(len(r)/2):
        name=people[d][0]
        origin=people[d][1]
        out=flights[(origin,destination)][r[d]]
        ret=flights[(destination,origin)][r[d+1]]
        print '%10s%10s %5s-%5s $%3s %5s-%5s $%3s' % (name,origin,
                                                    out[0],out[1],out[2],
                                                    ret[0],ret[1],ret[2])

In [6]:
s=[1,4,3,2,7,3,6,3,2,4,5,3]
printschedule(s)

   Seymour       BOS  8:04-10:11 $ 95 12:08-14:05 $142
    Franny       DAL 12:19-15:25 $342 10:51-14:16 $256
     Zooey       CAK 10:53-13:36 $189  9:58-12:56 $249
      Walt       MIA  9:15-12:29 $225 16:50-19:26 $304
     Buddy       ORD 16:43-19:00 $246 10:33-13:11 $132
       Les       OMA 11:08-13:07 $175 15:07-17:21 $129


In [7]:
def schedulecost(sol):
    totalprice=0
    latestarrival=0
    earliestdep=24*60
    for d in range(len(sol)/2):
        
        # Get the inbound and outbound flights
        origin=people[d][1]
        outbound=flights[(origin,destination)][int(sol[d])]
        returnf=flights[(destination,origin)][int(sol[d+1])]

        # Total price is the price of all outbound and return flights
        totalprice+=int(outbound[2])
        totalprice+=int(returnf[2])

        # Track the latest arrival and earliest departure
        if latestarrival<getminutes(outbound[1]): latestarrival=getminutes(outbound[1])
        if earliestdep>getminutes(returnf[0]): earliestdep=getminutes(returnf[0])
        
    # Every person must wait at the airport until the latest person arrives.
    # They also must arrive at the same time and wait for their flights.
    totalwait=0
    for d in range(len(sol)/2):
        origin=people[d][1]
        outbound=flights[(origin,destination)][int(sol[d])]
        returnf=flights[(destination,origin)][int(sol[d+1])]
        totalwait+=latestarrival-getminutes(outbound[1])
        totalwait+=getminutes(returnf[0])-earliestdep
    
    # Does this solution require an extra day of car rental? That'll be $50!
    if latestarrival>earliestdep: totalprice+=50
    return totalprice+totalwait


In [8]:
schedulecost(s)

5285

In [9]:
def randomoptimize(domain,costf):
    best=999999999
    bestr=None
    
    for i in range(10000):

        # Create a random solution
        r=[random.randint(domain[i][0],domain[i][1]) for i in range(len(domain))]

        # Get the cost
        cost=costf(r)

        # Compare it to the best one so far
        if cost<best:
            best=cost
            bestr=r
    return r

In [10]:
domain=[(0,8)]*(len(people)*2)
s=randomoptimize(domain,schedulecost)
schedulecost(s)
printschedule(s)

   Seymour       BOS 12:34-15:02 $109 12:08-14:05 $142
    Franny       DAL 12:19-15:25 $342 12:20-16:34 $500
     Zooey       CAK 12:08-14:59 $149 16:33-18:15 $253
      Walt       MIA 17:07-20:04 $291  6:33- 9:14 $172
     Buddy       ORD  6:05- 8:32 $174  6:03- 8:43 $219
       Les       OMA  6:11- 8:31 $249  6:19- 8:13 $239


In [11]:
def hillclimb(domain,costf):
    
    # Create a random solution
    sol=[random.randint(domain[i][0],domain[i][1]) for i in range(len(domain))]

    # Main loop
    while 1:

        # Create list of neighboring solutions
        neighbors=[]
        for j in range(len(domain)):

            # One away in each direction
            if sol[j]>domain[j][0]:
                neighbors.append(sol[0:j]+[sol[j]+1]+sol[j+1:])
            if sol[j]<domain[j][1]:
                neighbors.append(sol[0:j]+[sol[j]-1]+sol[j+1:])

        # See what the best solution amongst the neighbors is
        current=costf(sol)
        best=current
        for j in range(len(neighbors)):
            cost=costf(neighbors[j])
            if cost<best:
                best=cost
                sol=neighbors[j]

        # If there's no improvement, then we've reached the top
        if best==current:
            break
    return sol

In [12]:
best=hillclimb(domain,schedulecost)
schedulecost(s)
printschedule(s)

   Seymour       BOS 12:34-15:02 $109 12:08-14:05 $142
    Franny       DAL 12:19-15:25 $342 12:20-16:34 $500
     Zooey       CAK 12:08-14:59 $149 16:33-18:15 $253
      Walt       MIA 17:07-20:04 $291  6:33- 9:14 $172
     Buddy       ORD  6:05- 8:32 $174  6:03- 8:43 $219
       Les       OMA  6:11- 8:31 $249  6:19- 8:13 $239


In [13]:
def annealingoptimize(domain,costf,T=10000.0,cool=0.95,step=1):
    
    # Initialize the values randomly
    vec=[float(random.randint(domain[i][0],domain[i][1])) for i in range(len(domain))]
    
    while T>0.1:

        # Choose one of the indices
        i=random.randint(0,len(domain)-1)

        # Choose a direction to change it
        dir=random.randint(-step,step)

        # Create a new list with one of the values changed
        vecb=vec[:]
        vecb[i]+=dir
        if vecb[i]<domain[i][0]: vecb[i]=domain[i][0]
        elif vecb[i]>domain[i][1]: vecb[i]=domain[i][1]

        # Calculate the current cost and the new cost 
        ea=costf(vec)
        eb=costf(vecb)
        p=pow(math.e,(-eb-ea)/T)

        # Is it better, or does it make the probability # cutoff?
        if (eb<ea or random.random()<p):
            vec=vecb

        # Decrease the temperature
        T=T*cool
    
    vec = [int(n) for n in vec]
    return vec

In [14]:
s=annealingoptimize(domain,schedulecost)
schedulecost(s)
printschedule(s)

   Seymour       BOS 15:27-17:18 $151 10:33-12:03 $ 74
    Franny       DAL 10:30-14:57 $290 10:51-14:16 $256
     Zooey       CAK 10:53-13:36 $189 13:37-15:33 $142
      Walt       MIA 14:01-17:24 $338  9:25-12:46 $295
     Buddy       ORD  9:42-11:32 $169 15:04-17:23 $189
       Les       OMA 15:03-16:42 $135  9:31-11:43 $210


In [15]:
def geneticoptimize(domain,costf,popsize=50,step=1,mutprob=0.2,elite=0.2,maxiter=100):
    
    # Mutation Operation
    def mutate(vec):
        i=random.randint(0,len(domain)-1)
        if random.random()<0.5 and vec[i]>domain[i][0]:
            return vec[0:i]+[vec[i]-step]+vec[i+1:]
        elif vec[i]<domain[i][1]:
            return vec[0:i]+[vec[i]+step]+vec[i+1:]

    # Crossover Operation
    def crossover(r1,r2):
        i=random.randint(1,len(domain)-2)
        return r1[0:i]+r2[i:]

    # Build the initial population
    pop=[]
    for i in range(popsize):
        vec=[random.randint(domain[i][0],domain[i][1]) for i in range(len(domain))] 
        pop.append(vec)
    
    # How many winners from each generation?
    topelite=int(elite*popsize)
    
    # Main loop
    for i in range(maxiter):
        scores=[(costf(v),v) for v in pop] 
        scores.sort( )
        ranked=[v for (s,v) in scores]

        # Start with the pure winners
        pop=ranked[0:topelite]

        # Add mutated and bred forms of the winners
        while len(pop)<popsize:
            if random.random()<mutprob:

                # Mutation
                c=random.randint(0,topelite)
                pop.append(mutate(ranked[c]))
            else:

                # Crossover
                c1=random.randint(0,topelite)
                c2=random.randint(0,topelite)
                pop.append(crossover(ranked[c1],ranked[c2]))
    
        # Print current best score
        print scores[0][0]
    return scores[0][1]

In [19]:
s=geneticoptimize(domain,schedulecost)
printschedule(s)

4232
4232
4028
4028
3674
3674
3674
3644
3487
3303
3303
3273
3273
3273
3273
3273
3273
3273
3273
3273
3273
3273
3273
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
3069
   Seymour       BOS 12:34-15:02 $109 10:33-12:03 $ 74
    Franny       DAL 10:30-14:57 $290 10:51-14:16 $256
     Zooey       CAK 10:53-13:36 $189 10:32-13:16 $139
      Walt       MIA 11:28-14:40 $248  9:25-12:46 $295
     Buddy       ORD  9:42-11:32 $169 10:33-13:11 $132
       Les       OMA 11:08-13:07 $175  9:31-11:43 $210


In [20]:
import xml.dom.minidom
dom=xml.dom.minidom.parseString('<data><rec>Hello!</rec></data>')
dom

<xml.dom.minidom.Document instance at 0x1046bdfc8>

In [21]:
r=dom.getElementsByTagName('rec')
r

[<DOM Element: rec at 0x104728128>]

In [22]:
r[0].firstChild

<DOM Text node "u'Hello!'">

In [23]:
r[0].firstChild.data

u'Hello!'

In [24]:
import random
import math
# The dorms, each of which has two available spaces
dorms=['Zeus','Athena','Hercules','Bacchus','Pluto']
# People, along with their first and second choices
prefs=[('Toby', ('Bacchus', 'Hercules')),
        ('Steve', ('Zeus', 'Pluto')),
        ('Andrea', ('Athena', 'Zeus')),
        ('Sarah', ('Zeus', 'Pluto')),
        ('Dave', ('Athena', 'Bacchus')),
        ('Jeff', ('Hercules', 'Pluto')),
        ('Fred', ('Pluto', 'Athena')),
        ('Suzie', ('Bacchus', 'Hercules')),
        ('Laura', ('Bacchus', 'Hercules')),
        ('Neil', ('Hercules', 'Athena'))]

In [25]:
# [(0,9),(0,8),(0,7),(0,6),...,(0,0)]
domain=[(0,(len(dorms)*2)-i-1) for i in range(0,len(dorms)*2)]
domain

[(0, 9),
 (0, 8),
 (0, 7),
 (0, 6),
 (0, 5),
 (0, 4),
 (0, 3),
 (0, 2),
 (0, 1),
 (0, 0)]

In [26]:
def printsolution(vec):
    slots=[]
    # Create two slots for each dorm
    for i in range(len(dorms)): slots+=[i,i]
    # Loop over each students assignment
    for i in range(len(vec)):
        x=int(vec[i])
        # Choose the slot from the remaining ones
        dorm=dorms[slots[x]]
        # Show the student and assigned dorm
        print prefs[i][0],dorm
        # Remove this slot
        del slots[x]

In [27]:
printsolution([0,0,0,0,0,0,0,0,0,0])

Toby Zeus
Steve Zeus
Andrea Athena
Sarah Athena
Dave Hercules
Jeff Hercules
Fred Bacchus
Suzie Bacchus
Laura Pluto
Neil Pluto


In [40]:
def dormcost(vec):
    cost=0
    
    # Create list a of slots
    slots=[0,0,1,1,2,2,3,3,4,4]
    
    # Loop over each student
    for i in range(len(vec)):
        x=int(vec[i])
        dorm=dorms[slots[x]]
        pref=prefs[i][1]
        
        # First choice costs 0, second choice costs 1
        if pref[0]==dorm: cost+=0
        elif pref[1]==dorm: cost+=1
        else: cost+=3
            
        # Not on the list costs 3
        # Remove selected slot
        del slots[x]
    return cost

In [41]:
s=randomoptimize(domain,dormcost)
dormcost(s)

20

In [42]:
geneticoptimize(domain,dormcost)

12


TypeError: object of type 'NoneType' has no len()

In [43]:
printsolution(s)

Toby Hercules
Steve Zeus
Andrea Pluto
Sarah Athena
Dave Bacchus
Jeff Athena
Fred Bacchus
Suzie Zeus
Laura Pluto
Neil Hercules


In [44]:
import math
people=['Charlie','Augustus','Veruca','Violet','Mike','Joe','Willy','Miranda']
links=[('Augustus', 'Willy'),
    ('Mike', 'Joe'),
    ('Miranda', 'Mike'),
    ('Violet', 'Augustus'),
    ('Miranda', 'Willy'),
    ('Charlie', 'Mike'),
    ('Veruca', 'Joe'),
    ('Miranda', 'Augustus'),
    ('Willy', 'Augustus'),
    ('Joe', 'Charlie'),
    ('Veruca', 'Augustus'),
    ('Miranda', 'Joe')]